In [ ]:
import duckdb
import pandas as pd
import os

PAI_DIR = os.getcwd()  # diretório atual para notebooks jupyter
DATA_RAW = os.path.join(PAI_DIR, "data", "raw")
DATA_PROCESSED = os.path.join(PAI_DIR, "data", "processed")
SCRIPTS = os.path.join(PAI_DIR, "scripts")

pd.set_option("display.float_format", "{:,.0f}".format)

In [ ]:
!python {os.path.join(SCRIPTS, 'elt_execucao_orcamentaria.py')}

In [ ]:
con = duckdb.connect(database=':memory:')

In [ ]:
for arquivo in os.listdir(DATA_PROCESSED):
    if arquivo.startswith("estimativa_cre_") and arquivo.endswith(".parquet"):
        caminho = os.path.join(DATA_PROCESSED, arquivo)

        # Nome da tabela = nome do arquivo sem extensão
        nome_tabela = os.path.splitext(arquivo)[0]

        # Cria tabela no DuckDB (nome da tabela entre aspas duplas para aceitar hífens)
        con.execute(f"""
            CREATE TABLE IF NOT EXISTS "{nome_tabela}" AS 
            SELECT * FROM '{caminho}'
            """)
        print(f"✅ Tabela {nome_tabela} criada a partir de {arquivo}")

In [ ]:
tabelas = [
    "STG_SIGEF__FATO__EXECUCAO_ORCAMENTARIA.parquet",
    "STG_SIGEF__DIM__ACAO_PROGRAMA.parquet",
    "STG_SIGEF__DIM__CONTA_CONTABIL.parquet",
    "STG_SIGEF__DIM__CREDOR.parquet",
    "STG_SIGEF__DIM__DOMICILIO_BANCARIO.parquet",
    "STG_SIGEF__DIM__EVENTO.parquet",
    "STG_SIGEF__DIM__FONTE_RECURSO.parquet",
    "STG_SIGEF__DIM__GRUPO_PROGRAMACAO_FINANCEIRA.parquet",
    "STG_SIGEF__DIM__NATUREZA_DESPESA.parquet",
    "STG_SIGEF__DIM__NATUREZA_RECEITA.parquet",
    "STG_SIGEF__DIM__UNIDADE_GESTORA_GESTAO.parquet"
]

# Loop para criar as tabelas no DuckDB
for arquivo in tabelas:
    nome_tabela = os.path.splitext(arquivo)[0].lower()  # nome da tabela em minúsculo
    caminho = os.path.join(DATA_RAW, arquivo)
    con.execute(f"""
        CREATE TABLE IF NOT EXISTS {nome_tabela} AS
        SELECT * FROM '{caminho}'
    """)
    print(f"Tabela '{nome_tabela}' criada com sucesso!")

In [ ]:
df_est = con.execute("SELECT * FROM 'estimativa_cre_12-08-25' where metodo_principal = 'IPCA + PIB m'").df()

df_est 

In [ ]:
df_est_500 = con.execute('SELECT mes, SUM(valor) FROM "estimativa_cre_12-08-25" WHERE fonte_recurso = 500 and ano = 2025 group by mes').df()
df_est_500

In [ ]:
df_fato = con.execute("SELECT * FROM STG_SIGEF__FATO__EXECUCAO_ORCAMENTARIA LIMIT 2").df()
df_fato

In [ ]:
df_fonte = con.execute("SELECT * FROM stg_sigef__dim__fonte_recurso LIMIT 2").df()
df_fonte

In [ ]:
# Consulta inicial (pode ser LIMIT 10 para preview)
df_duck = con.execute("""
    SELECT 
        EXTRACT(MONTH FROM a.MES_REFERENCIA) AS MES,
        SUM(a.VALOR_RECEITA_PREVISTA_BRUTA_COTAS_MENSAIS+a.VALOR_RECEITA_PREVISTA_DEDUCOES_COTAS_MENSAIS) AS RECEITA_PREVISTA,
        SUM(a.VALOR_RECEITA_REALIZADA_BRUTA+a.VALOR_RECEITA_REALIZADA_DEDUCOES) AS RECEITA_REALIZADA
    FROM STG_SIGEF__FATO__EXECUCAO_ORCAMENTARIA a 
    JOIN stg_sigef__dim__fonte_recurso b 
        ON a.ID_FONTE_RECURSO = b.ID_FONTE_RECURSO
    WHERE b.FONTE_REDUZIDO = 500
        AND a.SIGEF_DB = 'SIGEF2025'
    GROUP BY a.MES_REFERENCIA
    ORDER BY EXTRACT(MONTH FROM a.MES_REFERENCIA)
""").df()

df_duck.head(13)

In [ ]:
df_teste = con.execute("""
    SELECT 
            mes AS MES,
            SUM(valor) AS RECEITA_REALIZADA_EST
        FROM "estimativa_cre_12-08-25"
        WHERE fonte_recurso = 500
            AND ano = 2025
        GROUP BY mes
    """).df()

df_teste

In [ ]:
df_final = con.execute("""
    WITH fato AS (
        SELECT 
            EXTRACT(MONTH FROM a.MES_REFERENCIA) AS MES,
            SUM(a.VALOR_RECEITA_PREVISTA_BRUTA_COTAS_MENSAIS + a.VALOR_RECEITA_PREVISTA_DEDUCOES_COTAS_MENSAIS) AS RECEITA_PREVISTA,
            SUM(a.VALOR_RECEITA_REALIZADA_BRUTA + a.VALOR_RECEITA_REALIZADA_DEDUCOES) AS RECEITA_REALIZADA_FATO
        FROM STG_SIGEF__FATO__EXECUCAO_ORCAMENTARIA a
        JOIN stg_sigef__dim__fonte_recurso b
            ON a.ID_FONTE_RECURSO = b.ID_FONTE_RECURSO
        WHERE b.FONTE_REDUZIDO = 500
            AND a.SIGEF_DB = 'SIGEF2025'
        GROUP BY EXTRACT(MONTH FROM a.MES_REFERENCIA)
    ),
    est AS (
        SELECT 
            mes AS MES,
            SUM(valor) AS RECEITA_REALIZADA_EST
        FROM "estimativa_cre_12-08-25"
        WHERE fonte_recurso = 500
            AND ano = 2025
        GROUP BY mes
    )
    SELECT
        f.MES,
        f.RECEITA_PREVISTA,
        CASE 
            WHEN f.MES >= 8 THEN e.RECEITA_REALIZADA_EST
            ELSE f.RECEITA_REALIZADA_FATO
        END AS RECEITA_REALIZADA,
        ((RECEITA_REALIZADA - RECEITA_PREVISTA) / RECEITA_PREVISTA) * 100 AS PERCENTUAL_REALIZADO,
        (RECEITA_REALIZADA - RECEITA_PREVISTA) AS DIFERENCA_VALOR
    FROM fato f
    LEFT JOIN est e
        ON f.MES = e.MES
    ORDER BY f.MES
""").df()


print("Tabela 2 - Previsão Atualizada x Receita Arrecadada/Projetada 2025 - Fonte 500")
df_final

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(df_final['MES'], df_final['RECEITA_PREVISTA'], marker='o', label='Receita Prevista', color='blue')
plt.plot(df_final['MES'], df_final['RECEITA_REALIZADA'], marker='o', label='Receita Realizada', color='orange')
plt.xlabel('Mês')
plt.ylabel('Valor (R$)')
plt.title('Receita Prevista vs Receita Realizada/Projetada - Fonte 500 (2025)')
plt.legend()
plt.show()

In [ ]:
df_final.plot(x='MES', y=['RECEITA_PREVISTA', 'RECEITA_REALIZADA'], kind='line', marker='o')
plt.show()

In [ ]:
import seaborn as sns

In [ ]:
sns.lineplot(data=df_final, x='MES', y='RECEITA_PREVISTA', marker='o', label='Receita Prevista')
sns.lineplot(data=df_final, x='MES', y='RECEITA_REALIZADA', marker='o', label='Receita Realizada')
plt.show()

In [ ]:
import plotly.express as px

In [ ]:
fig = px.line(df_final, x='MES', y=['RECEITA_PREVISTA', 'RECEITA_REALIZADA'], markers=True, title='Receita Prevista vs Receita Realizada/Projetada - Fonte 500 (2025)')
fig.show()

In [ ]:
import altair as alt

In [ ]:
df_melt = df_final.melt(id_vars=['MES'], value_vars=['RECEITA_PREVISTA', 'RECEITA_REALIZADA'], var_name='Tipo', value_name='Valor')

chart = alt.Chart(df_melt).mark_line(point=True).encode(
    x='MES:O',
    y=alt.Y("Valor", scale=alt.Scale(domain=[600_000_000, 1_400_000_000])),
    color='Tipo:N',
    tooltip=['MES', 'Tipo', 'Valor']
).interactive().properties(title='Receita Prevista vs Receita Realizada/Projetada - Fonte 500 (2025)')


chart.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_final['MES'], y=df_final['RECEITA_PREVISTA'], mode='lines+markers+text', name='Receita Prevista',
                         text=[f"{v:,.2f}" for v in df_final["RECEITA_PREVISTA"]],
    textposition="top center", line=dict(color="blue")))

fig.add_trace(go.Scatter(x=df_final['MES'], y=df_final['RECEITA_REALIZADA'], mode='lines+markers+text', name='Receita Realizada',
                            text=[f"{v:,.2f}" for v in df_final["RECEITA_REALIZADA"]],
        textposition="bottom center", line=dict(color="orange")))

fig.add_trace(go.Bar(x=df_final['MES'], y=df_final['PERCENTUAL_REALIZADO'], name='Variação %', marker_color='lightgrey', yaxis='y2'))

fig.update_layout(
    title='Receita Prevista vs Receita Realizada/Projetada - Fonte 500 (2025)',
    xaxis_title='Mês',
    yaxis_title='Valor (R$)',
    yaxis2=dict(
        title='Variação %',
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,255,255,0)', bordercolor='rgba(255,255,255,0)')
)

fig.show()

In [ ]:
df_final = con.execute("""
    WITH fato AS (
        SELECT 
            EXTRACT(MONTH FROM a.MES_REFERENCIA) AS MES,
            SUM(a.VALOR_RECEITA_PREVISTA_BRUTA_COTAS_MENSAIS + a.VALOR_RECEITA_PREVISTA_DEDUCOES_COTAS_MENSAIS) AS RECEITA_PREVISTA,
            SUM(a.VALOR_RECEITA_REALIZADA_BRUTA + a.VALOR_RECEITA_REALIZADA_DEDUCOES) AS RECEITA_REALIZADA_FATO
        FROM STG_SIGEF__FATO__EXECUCAO_ORCAMENTARIA a
        JOIN stg_sigef__dim__fonte_recurso b
            ON a.ID_FONTE_RECURSO = b.ID_FONTE_RECURSO
        WHERE b.FONTE_REDUZIDO = 500
            AND a.SIGEF_DB = 'SIGEF2025'
        GROUP BY EXTRACT(MONTH FROM a.MES_REFERENCIA)
    ),
    est AS (
        SELECT 
            mes AS MES,
            SUM(valor) AS RECEITA_REALIZADA_EST
        FROM "estimativa_cre_12-08-25"
        WHERE fonte_recurso = 500
            AND ano = 2025
        GROUP BY mes
    )
    SELECT
        f.MES,
        (f.RECEITA_PREVISTA) * .7495 AS RECEITA_PREVISTA,
        (CASE 
            WHEN f.MES >= 8 THEN e.RECEITA_REALIZADA_EST
            ELSE f.RECEITA_REALIZADA_FATO
        END) * .7495 AS RECEITA_REALIZADA,
        ((RECEITA_REALIZADA - RECEITA_PREVISTA) / RECEITA_PREVISTA) * 100 AS PERCENTUAL_REALIZADO,
        (RECEITA_REALIZADA - RECEITA_PREVISTA) AS DIFERENCA_VALOR
    FROM fato f
    LEFT JOIN est e
        ON f.MES = e.MES
    ORDER BY f.MES
""").df()


print("Tabela 3 - Previsão Atualizada x Receita Arrecadada/Projetada 2025 - Fonte 500 - Poder Executivo líquido duodécimos")
df_final

In [ ]:
from pandasgui import show

gui = show(df_final)


In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.getenv("HF_TOKEN"),
)

def consulta_ia(pergunta: str):
    prompt = f"""
    Você é um assistente que gera consultas SQL para DuckDB.
    A tabela disponível se chama 'estimativa_cre' e tem as seguintes colunas:
    {', '.join(df_duck.columns.tolist())}

    Pergunta do usuário: {pergunta}

    Responda apenas com a SQL.
    """

    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3-8B-Instruct",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=300,
        temperature=0,
    )

    query = response.choices[0].message.content.strip()

    # Executa a query no DuckDB
    try:
        resultado = con.execute(query).fetchdf()
        return query, resultado
    except Exception as e:
        return query, f"Erro ao executar SQL: {e}"

In [ ]:
query, resultado = consulta_ia("O total arrecadado por ano da fonte 500")
print("Query usada:\n", query)
print("\nResultado:\n", resultado)